## Movie recommendations using graph neural networks

Implementation of Inductive graph-based matrix completion (IGMC) from "M. Zhang and Y. Chen, Inductive Matrix Completion.

IGMC is an inductive matrix completion model based on graph neural networks without using any side information (latent or explicit features). In contrast to traditional matrix factorization approaches, IGMC does not require content to be known during training. The model achieves this by training a graph neural network (GNN) based purely on local subgraphs around (user, movie) pairs extracted from the bipartite graph formed by the rating matrix, and maps these subgraphs to their corresponding ratings. It does not rely on any global information specific to the rating matrix or the task, nor does it learn embeddings specific to the observed users/movies. Thus, IGMC is a completely inductive model. From this perspective, the recommendation task becomes a **link prediction problem**.

The model in this notebook is trained on the MovieLens dataset implementation from [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.MovieLens). Evaluation metric is RMSE.

This is an adaptation of https://github.com/muhanzhang/IGMC/tree/latest using PyTorch Geometric and Pythorch Lightning. 

In [2]:
import argparse
import os.path as osp
import torch.nn.functional as F

import torch
import numpy as np
import scipy.sparse as ssp

from tqdm import tqdm
from torchmetrics import MeanSquaredError
from pytorch_lightning import (LightningDataModule, LightningModule, Trainer,
                               seed_everything)
from pytorch_lightning.callbacks import ModelCheckpoint
from torch_geometric.nn import MLP, GCNConv, global_sort_pool
from torch_geometric.datasets import MovieLens
from torch_geometric.data import Dataset, Data
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import to_scipy_sparse_matrix 
from torch_geometric.loader import DataLoader


In [3]:
seed_everything(42)

42

In [4]:
class SparseRowIndexer:
    def __init__(self, csr_matrix):
        data = []
        indices = []
        indptr = []

        for row_start, row_end in zip(csr_matrix.indptr[:-1], csr_matrix.indptr[1:]):
            data.append(csr_matrix.data[row_start:row_end])
            indices.append(csr_matrix.indices[row_start:row_end])
            indptr.append(row_end - row_start)  # nnz of the row

        self.data = np.array(data, dtype=object)
        self.indices = np.array(indices, dtype=object)
        self.indptr = np.array(indptr, dtype=object)
        self.shape = csr_matrix.shape

    def __getitem__(self, row_selector):
        indices = np.concatenate(self.indices[row_selector])
        data = np.concatenate(self.data[row_selector])
        indptr = np.append(0, np.cumsum(self.indptr[row_selector]))
        shape = [indptr.shape[0] - 1, self.shape[1]]
        return ssp.csr_matrix((data, indices, indptr), shape=shape)


class SparseColIndexer:
    def __init__(self, csc_matrix):
        data = []
        indices = []
        indptr = []

        for col_start, col_end in zip(csc_matrix.indptr[:-1], csc_matrix.indptr[1:]):
            data.append(csc_matrix.data[col_start:col_end])
            indices.append(csc_matrix.indices[col_start:col_end])
            indptr.append(col_end - col_start)

        self.data = np.array(data, dtype=object)
        self.indices = np.array(indices, dtype=object)
        self.indptr = np.array(indptr, dtype=object)
        self.shape = csc_matrix.shape

    def __getitem__(self, col_selector):
        indices = np.concatenate(self.indices[col_selector])
        data = np.concatenate(self.data[col_selector])
        indptr = np.append(0, np.cumsum(self.indptr[col_selector]))

        shape = [self.shape[0], indptr.shape[0] - 1]
        return ssp.csc_matrix((data, indices, indptr), shape=shape)

In [5]:
R = ('user', 'rates', 'movie')
u = "user"
v = "movie"

In [6]:
class IGMCDataset(Dataset):
    def __init__(
            self, 
            root, 
            data: Data,
            h: int = 1, 
            max_nodes_per_hop: int = None, 
            max_num: int = None,
            use_features: bool = False
        ):
        super().__init__(root)
        
        self.labels = data[R].edge_label
        self.links = data[R].edge_label_index
        
        A = to_scipy_sparse_matrix(self.links, edge_attr=self.labels.float())        
        self.Arow = SparseRowIndexer(A.tocsr()) 
        self.Acol = SparseColIndexer(A.tocsc()) 

        self.h = h
        self.max_nodes_per_hop = max_nodes_per_hop

        self.u_features = None # data[u].num_nodes
        self.v_features = data[v].x
        self.class_values = np.sort(self.labels.unique())
        
        if max_num is not None:
            np.random.seed(123)
            num_links = len(self.links[0])
            perm = np.random.permutation(num_links)
            perm = perm[:max_num]
            self.links = (self.links[0][perm], self.links[1][perm])
            self.labels = self.labels[perm]

    def __len__(self):
        return len(self.links[0])

    def len(self):
        return self.__len__()
    
    def get(self, idx):
        i, j = self.links[0][idx], self.links[1][idx]
        g_label = self.labels[idx]
        graph_data = subgraph_extraction_labeling(
            (i, j), self.Arow, self.Acol, self.h, self.max_nodes_per_hop, 
            self.u_features, self.v_features, self.class_values, g_label
        )
        return construct_pyg_graph(*graph_data)

In [7]:
def neighbors(fringe, A):
    # find all 1-hop neighbors of nodes in fringe from A
    if not fringe:
        return set([])
    
    return set(A[list(fringe)].indices)

In [8]:
def one_hot(idx, length):
    idx = np.array(idx)
    x = np.zeros([len(idx), length])
    x[np.arange(len(idx)), idx] = 1.0
    return x

In [9]:
def links2subgraphs(Arow, 
                    Acol, 
                    links, 
                    labels, 
                    h=1, 
                    max_nodes_per_hop=None, 
                    u_features=None, 
                    v_features=None, 
                    class_values=None, 
                    parallel=True):
    
    with tqdm(total=len(links[0])) as pbar:

        g_list = []
        for i, j, g_label in zip(links[0], links[1], labels):
            graph_data = subgraph_extraction_labeling(
                ind=(i, j), 
                Arow=Arow, 
                Acol=Acol, 
                num_hops=1, 
                max_nodes_per_hop=max_nodes_per_hop,
                u_features=u_features, 
                v_features=v_features, 
                class_values=ratings, 
                y=g_label
            )
            data = construct_pyg_graph(*graph_data)
            g_list.append(data)
            pbar.update(1)
    
    return g_list

In [10]:
def subgraph_extraction_labeling(
    ind, 
    Arow, 
    Acol, 
    num_hops=1,
    max_nodes_per_hop=None,
    u_features=None, 
    v_features=None, 
    class_values=None, 
    y=0
):
    i, j = ind
    # extract the h-hop enclosing subgraph around link 'ind = i, j'
    u_nodes, v_nodes = [i], [j]
    u_dist, v_dist = [0], [0]
    u_visited, v_visited = set([i]), set([j])
    u_fringe, v_fringe = set([i]), set([j])
    
    for dist in range(1, num_hops + 1):
        v_fringe, u_fringe = neighbors(u_fringe, Arow), neighbors(v_fringe, Acol)
        u_fringe = u_fringe - u_visited
        v_fringe = v_fringe - v_visited
        u_visited = u_visited.union(u_fringe)
        v_visited = v_visited.union(v_fringe)
        
        if max_nodes_per_hop:
            if max_nodes_per_hop < len(u_fringe):
                u_fringe = random.sample(u_fringe, max_nodes_per_hop)
            if max_nodes_per_hop < len(v_fringe):
                v_fringe = random.sample(v_fringe, max_nodes_per_hop)
        if not (u_fringe and v_fringe):
            break
        
        u_nodes = u_nodes + list(u_fringe)
        v_nodes = v_nodes + list(v_fringe)
        u_dist = u_dist + [dist] * len(u_fringe)
        v_dist = v_dist + [dist] * len(v_fringe)
    
    # HACK: since the first item is stored as tensor, causes index out of range for dimension 2
    v_nodes[0] = v_nodes[0].item()
    
    subgraph = Arow[u_nodes][:, v_nodes]
    # remove link between target nodes
    subgraph[0, 0] = 0
        
    # prepare pyg graph constructor input
    u, v, r = ssp.find(subgraph)  # r is 0, 1, 2... (rating labels)
    v += len(u_nodes)
    # r = r - 1  # transform r back to rating label
    num_nodes = len(u_nodes) + len(v_nodes)
    node_labels = [x*2 for x in u_dist] + [x*2+1 for x in v_dist]
    max_node_label = 2 * num_hops + 1
    
    y = class_values[y]
    
    # get node features
    if u_features is not None:
        u_features = u_features[u_nodes]
    if v_features is not None:
        v_features = v_features[v_nodes]
    node_features = None
    
    # only output node features for the target user and item
    if u_features is not None and v_features is not None:
        node_features = [u_features[0], v_features[0]]
    
    return u, v, r, node_labels, max_node_label, y, node_features

In [11]:
def construct_pyg_graph(u, v, r, node_labels, max_node_label, y, node_features):
    u, v = torch.LongTensor(u), torch.LongTensor(v)
    r = torch.LongTensor(r)  
    edge_index = torch.stack([torch.cat([u, v]), torch.cat([v, u])], 0)
    edge_type = torch.cat([r, r])
    x = torch.FloatTensor(one_hot(node_labels, max_node_label+1))
    y = torch.FloatTensor([y])
    data = Data(x, edge_index, edge_type=edge_type, y=y)

    if node_features:
        if type(node_features) == list:  # a list of u_feature and v_feature
            u_feature, v_feature = node_features
            data.u_feature = torch.FloatTensor(u_feature).unsqueeze(0)
            data.v_feature = torch.FloatTensor(v_feature).unsqueeze(0)
        else:
            x2 = torch.FloatTensor(node_features)
            data.x = torch.cat([data.x, x2], 1)
    return data

In [12]:
path = osp.join(osp.dirname(osp.realpath("__file__")), '..', 'data', 'MovieLens')
dataset = MovieLens(path)
data = dataset[0]

Extracting /kaggle/data/MovieLens/raw/ml-latest-small.zip
Processing...


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Batches:   0%|          | 0/305 [00:00<?, ?it/s]

Done!


In [13]:
transform = RandomLinkSplit(
    num_val=0.05, 
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[R],
    is_undirected=True,
)
train_data, val_data, test_data = transform(data)

In [14]:
train_dataset = IGMCDataset(path, train_data)
val_dataset = IGMCDataset(path, val_data)
test_dataset = IGMCDataset(path, test_data)

In [15]:
class LightningIGMCDataset(LightningDataModule):
    def __init__(self,
                 train_dataset: Data,
                 val_dataset: Data,
                 test_dataset: Data,
                 batch_size: int = 1,
                 num_workers: int = 0,
                 **kwargs):
        super().__init__()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset
        self.batch_size = batch_size
        self.num_workers = num_workers
        
    def train_dataloader(self):
        return DataLoader(train_dataset, 
                          batch_size=self.batch_size, 
                          shuffle=True,
                          num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(val_dataset, 
                          batch_size=self.batch_size, 
                          shuffle=False,
                          num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(test_dataset, 
                          batch_size=self.batch_size, 
                          shuffle=False,
                          num_workers=self.num_workers)

In [16]:
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, Conv1d
from torch_geometric.nn import GCNConv, RGCNConv, global_sort_pool, global_add_pool
from torch_geometric.utils import dropout_adj
# from util_functions import *
import pdb
import time

class IGMC(torch.nn.Module):
    # The GNN model of Inductive Graph-based Matrix Completion. 
    # Use RGCN convolution + center-nodes readout.
    def __init__(self, dataset, gconv=RGCNConv, latent_dim=[32, 32, 32, 32], 
                 num_relations=5, num_bases=2, regression=False, adj_dropout=0.2, 
                 force_undirected=False, side_features=False, n_side_features=0, 
                 multiply_by=1):
        super().__init__()
        
        self.regression = regression
        self.adj_dropout = adj_dropout 
        self.force_undirected = force_undirected
        
        self.multiply_by = multiply_by
        self.convs = torch.nn.ModuleList()
        self.convs.append(gconv(dataset.num_features, latent_dim[0], num_relations, num_bases))
        for i in range(0, len(latent_dim)-1):
            self.convs.append(gconv(latent_dim[i], latent_dim[i+1], num_relations, num_bases))
        self.lin1 = Linear(2*sum(latent_dim), 128)
        self.side_features = side_features
        if side_features:
            self.lin1 = Linear(2*sum(latent_dim)+n_side_features, 128)

        if self.regression:
            self.lin2 = Linear(128, 1)
        else:
            self.lin2 = Linear(128, dataset.num_classes)
            
    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()
        
    def forward(self, data):

        x, edge_index, edge_type, batch = data.x, data.edge_index, data.edge_type, data.batch
        if self.adj_dropout > 0:
            edge_index, edge_type = dropout_adj(
                edge_index, edge_type, p=self.adj_dropout, 
                force_undirected=self.force_undirected, num_nodes=len(x), 
                training=self.training
            )
        concat_states = []
        for conv in self.convs:
            x = torch.tanh(conv(x, edge_index, edge_type))
            concat_states.append(x)
        concat_states = torch.cat(concat_states, 1)

        # TODO: dobbeltsjekk
        users = data.x[:, 0] == 1
        items = data.x[:, 1] == 1
        x = torch.cat([concat_states[users], concat_states[items]], 1)
        if self.side_features:
            x = torch.cat([x, data.u_feature, data.v_feature], 1)

        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        if self.regression:
            return x[:, 0] * self.multiply_by
        else:
            return F.log_softmax(x, dim=-1)

In [17]:
def num_graphs(data):
    if data.batch is not None:
        return data.num_graphs
    else:
        return data.x.size(0)
    
class LightningIGMC(LightningModule):
    def __init__(self, 
                 lr, 
                 weight_decay, 
                 lr_decay_factor,
                 lr_decay_step_size,
                 ARR=0, 
                 regression=False, 
                 **model_params):
        super().__init__()
        self.lr = lr
        self.weight_decay = weight_decay
        self.lr_decay_factor=lr_decay_factor
        self.lr_decay_step_size=lr_decay_step_size
        self.ARR = ARR
        self.regression = regression
        
        self.model = IGMC(regression=regression, **model_params)
        
        self.train_rmse = MeanSquaredError(squared=False)
        self.val_rmse = MeanSquaredError(squared=False)
        self.test_rmse = MeanSquaredError(squared=False)
        
        if self.regression:
            self.criterion = F.mse_loss
        else:
            self.criterion = F.nll_loss

    def forward(self, data):
        return self.model(data)

    def training_step(self, batch, batch_idx: int):
        out = self(batch)
        loss = self.criterion(out, batch.y.view(-1))
        
        if self.ARR != 0:
            for gconv in self.model.convs:
                w = torch.matmul(
                    gconv.comp, 
                    gconv.weight.view(gconv.num_bases, -1)
                ).view(gconv.num_relations, gconv.in_channels, gconv.out_channels)
                reg_loss = torch.sum((w[1:, :, :] - w[:-1, :, :])**2)
                loss += self.ARR * reg_loss
        
                
        self.train_rmse(out, batch.y.view(-1)) 
        
        self.log('train_rmse', self.train_rmse, on_step=True, on_epoch=True,
                 prog_bar=True, sync_dist=True)
        
        return loss * num_graphs(batch)  # TODO: remove?

    def validation_step(self, batch, batch_idx: int):
        out = self(batch)
        loss = self.criterion(out, batch.y.view(-1), reduction="sum")
        
        self.val_rmse(out, batch.y.view(-1)) 
        
        self.log('val_rmse', self.val_rmse, on_step=False, on_epoch=True,
                 prog_bar=True, sync_dist=True)
        
    def test_step(self, batch, batch_idx: int):
        out = self(batch)
        loss = self.criterion(out, batch.y.view(-1), reduction="sum")
        
        self.test_rmse(out, batch.y.view(-1)) 
        
        self.log('test_rmse', self.test_rmse, on_step=False, on_epoch=True,
                 prog_bar=True, sync_dist=True)
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=self.lr_decay_step_size, gamma=self.lr_decay_factor)
        return [optimizer], [scheduler]
    
    def parameters(self):
        return self.model.parameters()

In [18]:
parser = argparse.ArgumentParser(description='IGMC')

parser.add_argument('--adj-dropout', type=float, default=0.2, 
                    help='if not 0, random drops edges from adjacency matrix with this prob')
parser.add_argument('--force-undirected', action='store_true', default=False, 
                    help='in edge dropout, force (x, y) and (y, x) to be dropped together')
parser.add_argument('--use_features', action='store_true', 
                    help="whether to use raw node features as additional GNN input")

parser.add_argument('--batch_size', type=int, default=50)
parser.add_argument('--lr', type=float, default=1e-3, metavar='LR',
                    help='learning rate (default: 1e-3)')
parser.add_argument('--lr-decay-step-size', type=int, default=50,
                    help='decay lr by factor A every B steps')
parser.add_argument('--lr-decay-factor', type=float, default=0.1,
                    help='decay lr by factor A every B steps')
parser.add_argument('--weight_decay', type=float, default=0)  
parser.add_argument('--ARR', type=float, default=0.001)
parser.add_argument('--epochs', type=int, default=50)

parser.add_argument('--devices', type=int, default=1)
parser.add_argument('--num_workers', type=int, default=4)

args = parser.parse_args("")
print(args)

Namespace(ARR=0.001, adj_dropout=0.2, batch_size=50, devices=1, epochs=50, force_undirected=False, lr=0.001, lr_decay_factor=0.1, lr_decay_step_size=50, num_workers=4, use_features=False, weight_decay=0)


In [19]:
datamodule = LightningIGMCDataset(
    train_dataset,
    val_dataset,
    test_dataset,
    batch_size=args.batch_size, 
    num_workers=args.num_workers,
    use_features=args.use_features
)

In [20]:
num_relations = len(data[R].edge_label.unique())
multiply_by = 1

if args.use_features:
    u_features, v_features = train_dataset.u_features.toarray(), train_dataset.v_features.toarray()
    n_features = u_features.shape[1] + v_features.shape[1]
    print('Number of user features {}, movie features {}, total features {}'.format(
        u_features.shape[1], v_features.shape[1], n_features))
else:
    u_features, v_features = None, None
    n_features = 0
    
model = LightningIGMC( 
    lr=args.lr,
    weight_decay=args.weight_decay,
    lr_decay_step_size=args.lr_decay_step_size,
    lr_decay_factor=args.lr_decay_factor,
    ARR=args.ARR,
    regression=True, 
    dataset=train_dataset, 
    latent_dim=[32, 32, 32, 32], 
    num_relations=num_relations, 
    num_bases=4, 
    adj_dropout=args.adj_dropout, 
    force_undirected=args.force_undirected, 
    side_features=args.use_features, 
    n_side_features=n_features, 
    multiply_by=multiply_by
)

print(f'#Params {sum([p.numel() for p in model.parameters()])}')

#Params 49249


In [21]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_rmse', 
    mode='min',
    save_top_k=1
)
trainer = Trainer(
    accelerator="auto", 
    devices=args.devices, 
    max_epochs=args.epochs,      
    callbacks=[checkpoint_callback],
    default_root_dir=f'logs/IGMC'
)

In [22]:
trainer.fit(model, datamodule)

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Training: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Validation: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Validation: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 